# Helper tool to create the chatbot NLP model

This small program is used to create the model for the weather chatbot.
It loads the definitions for possible questions and answers, preprocess them to get the tags and words, split it into a training and a test set and generate the model. There is a small problem, because the number of data is really small and therefore the splitt into training and test data is not optimal. But for a first test it works and could later be optimized.

# Initialisation Part
First we load the necessary libraries. Also the os and inspect library to get the actual directory which is not automatically the same as the directory Python uses.
For a description of the other libraries look at the main Jupyter notebook NLP-Project.ipynb.

In [1]:
# Import of the NLP necessary libraries
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

# import the library to get the script directory
import os
import inspect

# Properly determining the current script directory for python file only
script_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) 
# Now the script directory is known

# We have a Jupyter notebook. So we can use relative paths.
script_dir = "."

# initialisation of some data objects
words        = []
classes      = []
documents    = []
ignore_words = ['?', '!']
# load the definitions from the json file
data_file    = open(script_dir + '/intents.json').read()
intents      = json.loads(data_file)



# Part 2 Load and prepare training data

In the next steps the definition data is loaded from the definition file. It is split into tagged lemmatized words and written back into files.

From the results a random train and testset is created.

In [2]:
# from the intents.json file, the data objects are filled
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# Write the word and classes results into files
pickle.dump(words,open(script_dir + '/words.pkl','wb'))
pickle.dump(classes,open(script_dir + '/classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
# with the new numpy version , dtype=object must be added
training = np.array(training, dtype=object)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")




49 documents
8 classes ['date_search', 'global_question', 'goodbye', 'greeting', 'location_search', 'number_search', 'options', 'thanks']
78 unique lemmatized words ["'s", ',', '2023/08/21', '2023/08/25', '21.08.2023', '25.08.2023', 'any', 'anyone', 'are', 'at', 'awesome', 'be', 'bye', 'can', 'chatting', 'cold', 'could', 'day', 'do', 'experience', 'for', 'forecast', 'four', 'from', 'give', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'hot', 'how', 'i', 'in', 'is', 'it', 'later', 'like', 'me', 'next', 'nice', 'offered', 'or', 'outside', 'please', 'provide', 'rain', 'raining', 'see', 'shining', 'snow', 'sun', 'support', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'three', 'til', 'till', 'time', 'to', 'today', 'tomorrow', 'two', 'until', 'weather', 'week', 'what', 'will', 'would', 'you']
Training data created


# Part 3 create model

In the next steps, the model is created and the model is saved to use at the NLP-Project Chatbot.
The model definitions are parts of the nltk library but under that the library tensorflow is used.

In [3]:

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# With the new tensorflow version you have to replace lr=0.01, decay=1e-6 with learning_rate=0.001
sgd = SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save(script_dir + '/chatbot_model.h5', hist)

print("model created")

Epoch 1/200
10/10 [==============================] - 0s 749us/step - loss: 2.1086 - accuracy: 0.1020
Epoch 2/200
10/10 [==============================] - 0s 670us/step - loss: 2.1245 - accuracy: 0.0408
Epoch 3/200
10/10 [==============================] - 0s 562us/step - loss: 2.0915 - accuracy: 0.1429
Epoch 4/200
10/10 [==============================] - 0s 590us/step - loss: 2.0285 - accuracy: 0.2449
Epoch 5/200
10/10 [==============================] - 0s 565us/step - loss: 2.0470 - accuracy: 0.1429
Epoch 6/200
10/10 [==============================] - 0s 567us/step - loss: 2.0629 - accuracy: 0.1224
Epoch 7/200
10/10 [==============================] - 0s 623us/step - loss: 2.0868 - accuracy: 0.1224
Epoch 8/200
10/10 [==============================] - 0s 563us/step - loss: 2.0664 - accuracy: 0.1837
Epoch 9/200
10/10 [==============================] - 0s 588us/step - loss: 2.0173 - accuracy: 0.2245
Epoch 10/200
10/10 [==============================] - 0s 538us/step - loss: 2.0609 - accura

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
